In [1]:
import tensorflow as tf
import numpy as np
import cv2
import time
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import os

2023-05-13 14:13:54.848836: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-13 14:13:55.430157: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-13 14:13:55.650008: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
%matplotlib inline

In [3]:
# SSD with Mobilenet v2

model_handle = 'https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2'

In [4]:
hub_model = hub.load(model_handle)

2023-05-13 14:14:03.579021: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-13 14:14:03.625364: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-13 14:14:03.625866: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-13 14:14:03.628011: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the ap

In [5]:
folder_path = 'data/inputs/'

In [6]:
input_folders = os.listdir(folder_path)
input_folders

['others', 'zomato']

In [12]:
# Loop through each folder and write output to output folder.

for folder in input_folders:
    images = os.listdir(os.path.join(folder_path ,folder))
    # filter .jpg images
    images = [x for x in images if x.endswith('.jpg')]
    
    # for each image find the objects and write to output folder.
    for image_name in images:
        image = cv2.imread(folder_path + folder + '/' + image_name)
        print("processing image: ", folder_path , folder , '/' , image_name)
        
        if image is not None:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, (320, 320))
            image = image[np.newaxis, ...]
            # image = tf.image.convert_image_dtype(image, tf.float32)[tf.newaxis, ...]
            start_time = time.time()
            result = hub_model(image)
            end_time = time.time()
            result = {key:value.numpy()[0] for key,value in result.items()}
            
            # filter results where detection_classes == 1
            human_idx = np.where(result['detection_classes'] == 1)

            human_results = {}
            human_results['detection_boxes'] = result['detection_boxes'][human_idx]
            human_results['detection_scores'] = result['detection_scores'][human_idx]
            
            for i in range(len(human_results['detection_boxes'])):
                box = human_results['detection_boxes'][i]
                box = [int(x * 320) for x in box]
                conf_score = human_results['detection_scores'][i]
                if conf_score > 0.5:
                    # write to output folder as image
                    cropped_img = image[0][box[0]:box[2], box[1]:box[3]]
                    if (cropped_img.shape[0] > 10) and (cropped_img.shape[1] > 10):
                        cropped_img = cv2.cvtColor(cropped_img, cv2.COLOR_RGB2BGR)
                        cv2.imwrite('data/outputs/' + folder + '/' + image_name + '_' + str(i) + '.jpg', cropped_img)
            
    
    

        

processing image:  data/inputs/ others / 1000.RID02509.2.jpg
processing image:  data/inputs/ others / 1170px-Dunzo-Char-Bangla-Andheri-BESTpedia.jpg
processing image:  data/inputs/ others / 1645204759-5501.jpg
processing image:  data/inputs/ others / 18175-zomatosjvccircle.jpg
processing image:  data/inputs/ others / 191122070328-01-dunzo-app-medium-plus-169.jpg
processing image:  data/inputs/ others / 70987351.jpg
processing image:  data/inputs/ others / 99542.DunzoKabeerBiswas.jpg
processing image:  data/inputs/ others / a-2022-05-06T120511.315.jpg
processing image:  data/inputs/ others / afaqs2F2021-052F6cc2681e-d9a6-45d6-8137-5f4e45dce4ea2FSai_Ganesh___Head___Marketing___Brand.JPG.jpg
processing image:  data/inputs/ others / AOLn63Gisl_gICLuje0rvR_u_4c7rIuatUteMYRIuC4mxQs64-c-mo.jpg
processing image:  data/inputs/ others / bttb2-11_400x.jpg
processing image:  data/inputs/ others / bttb2-3_400x.jpg
processing image:  data/inputs/ others / career-page-banner-mobile-fde7e062ac00bab564

In [ ]:
image[0][box[0]:box[2], box[1]:box[3]]

In [ ]:
image.shape

In [ ]:
img = cv2.imread('data/test2.jpg')
img.shape

In [ ]:
# resize image to 320x320
img = cv2.resize(img, (320, 320))

In [ ]:
img[np.newaxis, ...].shape

In [ ]:
results = hub_model(img[np.newaxis, ...])

In [ ]:


# Filter human class and get bounding boxes
# Filter entries which have detection_classes have 1
result = {key:value.numpy()[0] for key,value in results.items()}


In [ ]:
# filter results where detection_classes == 1
human_idx = np.where(result['detection_classes'] == 1)

human_results = {}
human_results['detection_boxes'] = result['detection_boxes'][human_idx]
human_results['detection_scores'] = result['detection_scores'][human_idx]


In [ ]:
# plot bounding boxes
for i in range(len(human_results['detection_boxes'])):
    box = human_results['detection_boxes'][i]
    box = [int(x * 320) for x in box]
    conf_score = human_results['detection_scores'][i]
    if conf_score > 0.6:
        img = cv2.rectangle(img, (box[1], box[0]), (box[3], box[2]), (0, 255, 0), 2)
    

In [ ]:
plt.imshow(img)